In [1]:
# !pip install pandas requests beautifulsoup4 webdriver_manager selenium

In [2]:
### VALORES PESQUISADOS

LISTA_TERMOS = [
    "Carnibol baunilha"
]

CEP = 71925000

In [ ]:
### PESQUISA E GERAÇÃO DE RESULTADOS PRELIMINARES
import time
import random
import re
import pandas as pd
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# ==============================================================================
# 1. CONFIGURAÇÕES
# ==============================================================================

# Defina a sua lista de termos aqui ou em uma célula anterior
# Exemplo: LISTA_TERMOS = ["tapete higienico cachorro", "tapete sanitário cães"] 
if 'LISTA_TERMOS' not in locals():
    LISTA_TERMOS = ["tapete higienico"] # Fallback caso não tenha definido

META_POR_TERMO = 50
ARQUIVO_SAIDA = "resultado_tapetes_final.csv"

# ==============================================================================
# 2. FUNÇÕES DE SUPORTE E CSV
# ==============================================================================
def limpar_preco(texto):
    if not texto: return 0.0
    apenas_numeros = re.sub(r'[^\d,]', '', texto)
    apenas_numeros = apenas_numeros.replace(',', '.')
    try:
        return float(apenas_numeros)
    except:
        return 0.0

def extrair_quantidade(titulo):
    titulo = titulo.lower()
    mult = re.search(r'(\d+)\s*(?:pcts|pct|pacotes|cx|caixas|x)\s*(?:de|x)?\s*(\d+)\s*(?:un|uni|u|toalhas)', titulo)
    if mult:
        return int(mult.group(1)) * int(mult.group(2))
    simples = re.search(r'(\d+)\s*(?:un|uni|und|unidades|tapetes)', titulo)
    if simples:
        return int(simples.group(1))
    return 1

def inicializar_csv():
    """Cria o arquivo CSV novo e escreve o cabeçalho, apagando dados antigos."""
    colunas = ['Termo', 'Título', 'Preço Total', 'Qtd', 'Valor Unitário', 'Frete Tipo', 'Valor Frete', 'Link']
    df_vazio = pd.DataFrame(columns=colunas)
    df_vazio.to_csv(ARQUIVO_SAIDA, index=False, sep=';', encoding='utf-8-sig')
    print(f"📁 Arquivo '{ARQUIVO_SAIDA}' inicializado (dados antigos apagados).")

def salvar_incremental(novos_dados):
    """Adiciona novos dados ao CSV existente sem apagar o que já está lá."""
    if not novos_dados: return
    
    df_novo = pd.DataFrame(novos_dados)
    
    # O modo 'a' significa append (adicionar ao final)
    # header=False para não repetir o cabeçalho a cada salvamento
    df_novo.to_csv(ARQUIVO_SAIDA, mode='a', index=False, sep=';', encoding='utf-8-sig', header=False)
    # print(f"   💾 +{len(novos_dados)} itens salvos no CSV.")

# ==============================================================================
# 3. MOTOR DO NAVEGADOR
# ==============================================================================
def iniciar_driver():
    print("🔧 Configurando navegador...")
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized") 
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled") 
    chrome_options.add_argument("--log-level=3")
    
    try:
        driver = webdriver.Chrome(options=chrome_options)
        return driver
    except Exception as e:
        print(f"❌ Erro ao abrir Chrome: {e}")
        return None

def buscar_por_termo(driver, termo, links_visitados):
    count_termo = 0
    termo_url = termo.replace(' ', '-')
    offset = 0
    
    print(f"\n🔎 BUSCANDO: '{termo}'")
    
    while count_termo < META_POR_TERMO:
        dados_pagina = [] # Buffer temporário para salvar apenas esta página
        
        if offset == 0:
            url = f"https://lista.mercadolivre.com.br/{termo_url}"
        else:
            url = f"https://lista.mercadolivre.com.br/{termo_url}_Desde_{offset + 1}"
            
        print(f"   ...Pagina {offset//50 + 1} (Coletados neste termo: {count_termo})")
        
        try:
            driver.get(url)
            time.sleep(random.uniform(2, 4))
            
            # Rolagem para carregar imagens e scripts
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight/3);")
            time.sleep(1)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight/1.5);")
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            produtos_html = soup.find_all('li', class_='ui-search-layout__item')
            if not produtos_html:
                produtos_html = soup.find_all('div', class_='poly-card')
            if not produtos_html:
                produtos_html = soup.find_all('div', class_='ui-search-result__wrapper')

            if not produtos_html:
                print("   ⚠️ Fim dos resultados ou página vazia.")
                break

            for produto in produtos_html:
                try:
                    # Link e verificação de duplicidade
                    link_tag = produto.find('a')
                    link = link_tag['href'] if link_tag else "N/A"
                    
                    # Se já visitamos este link nesta rodada, ignora
                    if link in links_visitados:
                        continue
                    
                    # Título
                    titulo_tag = produto.find('h2', class_='poly-box')
                    if not titulo_tag: titulo_tag = produto.find('h2', class_='ui-search-item__title')
                    if not titulo_tag: titulo_tag = produto.find('a', class_='poly-component__title')
                    if not titulo_tag: continue
                    titulo = titulo_tag.text.strip()
                    
                    # Preço
                    preco = 0.0
                    price_new = produto.find('div', class_='poly-price__current')
                    if price_new:
                        elem_preco = price_new.find('span', class_='andes-money-amount__fraction')
                        if elem_preco: preco = limpar_preco(elem_preco.text)
                    
                    if preco == 0:
                        price_old = produto.find('div', class_='ui-search-price__second-line')
                        if price_old:
                            elem_preco = price_old.find('span', class_='andes-money-amount__fraction')
                            if elem_preco: preco = limpar_preco(elem_preco.text)
                    
                    if preco == 0: continue

                    # --- LÓGICA DE FRETE MELHORADA ---
                    texto_completo = produto.text.lower()
                    tipo_frete = "Normal"
                    valor_frete = "A Calcular" # Default
                    
                    if "full" in texto_completo:
                        tipo_frete = "Full"
                    
                    if "grátis" in texto_completo or "gratis" in texto_completo:
                        if tipo_frete == "Full":
                            tipo_frete = "Full + Grátis"
                        else:
                            tipo_frete = "Grátis"
                        valor_frete = "0.0"

                    # Cálculos
                    qtd = extrair_quantidade(titulo)
                    unitario = preco / qtd if qtd > 0 else 0
                    
                    if qtd == 1 and unitario > 15: continue 

                    # Adiciona à lista temporária da página
                    dados_pagina.append({
                        'Termo': termo,
                        'Título': titulo,
                        'Preço Total': preco,
                        'Qtd': qtd,
                        'Valor Unitário': round(unitario, 3),
                        'Frete Tipo': tipo_frete,
                        'Valor Frete': valor_frete,
                        'Link': link
                    })
                    
                    # Marca como visitado
                    links_visitados.add(link)
                    
                except Exception:
                    continue
            
            # --- SALVAMENTO INCREMENTAL (Fim do loop de produtos da página) ---
            if dados_pagina:
                salvar_incremental(dados_pagina)
                count_termo += len(dados_pagina)
            else:
                # Se não achou nada válido na página, pode ser hora de parar
                print("   ⚠️ Nenhum produto válido nesta página.")

            offset += 50
            
        except Exception as e:
            print(f"Erro na página: {e}")
            break
            
    return count_termo

# ==============================================================================
# 4. EXECUÇÃO
# ==============================================================================
def main():
    # 1. Preparar CSV
    inicializar_csv()
    
    # 2. Iniciar navegador
    driver = iniciar_driver()
    if not driver: return

    # 3. Conjunto para evitar duplicatas globais na execução
    links_visitados = set()
    total_coletado = 0
    
    try:
        for termo in LISTA_TERMOS:
            qtd = buscar_por_termo(driver, termo, links_visitados)
            total_coletado += qtd
            print(f"✅ Fim de '{termo}'.")
            time.sleep(2)
            
    except KeyboardInterrupt:
        print("\n🛑 Parada forçada pelo usuário.")
    finally:
        driver.quit()
        print("\n" + "="*50)
        print(f"🏁 Processo finalizado.")
        print(f"📊 Total de itens salvos: {total_coletado}")
        print(f"📁 Dados disponíveis em: {ARQUIVO_SAIDA}")

if __name__ == "__main__":
    main()

📁 Arquivo 'resultado_tapetes_final.csv' inicializado (dados antigos apagados).
🔧 Configurando navegador...

🔎 BUSCANDO: 'Carnibol baunilha'
   ...Pagina 1 (Coletados neste termo: 0)
   ⚠️ Nenhum produto válido nesta página.
   ...Pagina 2 (Coletados neste termo: 0)
   ⚠️ Nenhum produto válido nesta página.
   ...Pagina 3 (Coletados neste termo: 0)
   ⚠️ Nenhum produto válido nesta página.
   ...Pagina 4 (Coletados neste termo: 0)
   ⚠️ Nenhum produto válido nesta página.
   ...Pagina 5 (Coletados neste termo: 0)
   ⚠️ Nenhum produto válido nesta página.
   ...Pagina 6 (Coletados neste termo: 0)
   ⚠️ Nenhum produto válido nesta página.
   ...Pagina 7 (Coletados neste termo: 0)


In [ ]:
### PESQUISE COM FILTROS NO DF

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# ==============================================================================
# 1. CARREGAR DADOS (Recupera o que o robô salvou)
# ==============================================================================
NOME_ARQUIVO = "resultado_tapetes_final.csv"

try:
    # Lê o arquivo CSV separado por ponto e vírgula
    df = pd.read_csv(NOME_ARQUIVO, sep=';')
    print(f"✅ Sucesso! {len(df)} anúncios carregados do arquivo '{NOME_ARQUIVO}'.")
except FileNotFoundError:
    print(f"❌ ERRO: O arquivo '{NOME_ARQUIVO}' não foi encontrado.")
    print("Certifique-se de que rodou a célula do Robô de Busca primeiro.")
    df = pd.DataFrame() # Cria vazio para não dar erro no resto do código

# ==============================================================================
# 2. PAINEL DE FILTRAGEM
# ==============================================================================

# Função para tornar o link clicável novamente (CSVs salvam apenas texto)
def make_clickable(val):
    if val and str(val).startswith('http'):
        return f'<a target="_blank" href="{val}">Ver Anúncio ↗</a>'
    return val

def mostrar_painel_filtragem():
    if df.empty: return

    print("\n📊 PAINEL DE ANÁLISE INTERATIVO")
    print("Filtre os resultados por palavras-chave no título:")

    # --- Widgets (Caixas de Texto) ---
    style = {'description_width': 'initial'}
    
    caixa_incluir = widgets.Text(
        placeholder='Ex: carvão, 30 unidades',
        description='✅ Título CONTÉM:',
        style=style,
        layout=widgets.Layout(width='450px')
    )
    
    caixa_excluir = widgets.Text(
        placeholder='Ex: defeito, kit 10',
        description='🚫 Título NÃO TEM:',
        style=style,
        layout=widgets.Layout(width='450px')
    )
    
    # Botão para limpar filtros
    botao_limpar = widgets.Button(description="Limpar Filtros", icon="eraser")
    
    output = widgets.Output()

    # --- Função de Atualização ---
    def filtrar_e_mostrar(change=None):
        termo_incluir = caixa_incluir.value.lower()
        termo_excluir = caixa_excluir.value.lower()
        
        df_filtrado = df.copy()
        
        # 1. Filtro de Inclusão
        if termo_incluir:
            df_filtrado = df_filtrado[df_filtrado['Título'].str.lower().str.contains(termo_incluir, na=False)]
            
        # 2. Filtro de Exclusão
        if termo_excluir:
            df_filtrado = df_filtrado[~df_filtrado['Título'].str.lower().str.contains(termo_excluir, na=False)]
        
        # Atualiza a saída
        with output:
            clear_output(wait=True)
            if len(df_filtrado) > 0:
                print(f"Encontrados: {len(df_filtrado)} anúncios.")
                
                # Selecionar e ordenar colunas
                cols = ['Título', 'Qtd', 'Preço Total', 'Valor Unitário', 'Frete', 'Link']
                # Garante que as colunas existem
                cols_finais = [c for c in cols if c in df_filtrado.columns]
                
                # Exibe tabela formatada (sem cortar o texto do título)
                display(
                    df_filtrado[cols_finais]
                    .head(20) # Mostra os top 20 do filtro
                    .style
                    .format({'Link': make_clickable})
                    .set_properties(subset=['Título'], **{'text-align': 'left', 'white-space': 'normal'})
                )
            else:
                print("⚠️ Nenhum resultado encontrado com esses filtros.")

    def limpar_filtros(b):
        caixa_incluir.value = ""
        caixa_excluir.value = ""

    # Conectar ações
    caixa_incluir.observe(filtrar_e_mostrar, names='value')
    caixa_excluir.observe(filtrar_e_mostrar, names='value')
    botao_limpar.on_click(limpar_filtros)

    # Mostrar layout
    display(widgets.HBox([caixa_incluir, caixa_excluir, botao_limpar]))
    display(output)
    
    # Primeira execução
    filtrar_e_mostrar()

# Executa
mostrar_painel_filtragem()